In [1]:
# Import the Twython class
from twython import Twython
import json
import pandas as pd

In [2]:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

In [3]:
# Instantiate an object using personal API key credentials
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

In [4]:
tweets = []
MAX_ATTEMPTS = 150
COUNT_OF_TWEETS_TO_BE_FETCHED = 15000
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': [], 'hashtags' : []}

for i in range(0,MAX_ATTEMPTS):

    if(COUNT_OF_TWEETS_TO_BE_FETCHED < len(dict_)):
        break


    # STEP 1: Query Twitter
    if(0 == i):
        # Query twitter for data. 
        results = python_tweets.search(q="#stocks", until = '2020-08-11',count='100', result_type = 'recent')
    else:
        # After the first call we should have max_id from result of previous call. Pass it in query.
        results = python_tweets.search(q="#stocks",count ='100',include_entities='true',max_id=next_max_id,result_type = 'recent')

    # STEP 2: Save the returned tweets
    for result in results['statuses']:
        dict_['user'].append(result['user']['screen_name'])
        dict_['date'].append(result['created_at'])
        dict_['text'].append(result['text'])
        dict_['favorite_count'].append(result['favorite_count'])
        dict_['hashtags'].append(result['entities']['hashtags'])


    # STEP 3: Get the next max_id
    try:
        # Parse the data returned to get max_id to be passed in consequent call.
        next_results_url_params = results['search_metadata']['next_results']
        next_max_id = next_results_url_params.split('max_id=')[1].split('&')[0]
    except:
        # No more next pages
        break

In [5]:
# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.to_csv('tweets08-2.csv')